In [92]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import dash_bootstrap_components as dbc
import pandas as pd
from sklearn.linear_model import LinearRegression
import dash_bootstrap_components as dbc
import locale
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from prophet import Prophet
import numpy as np
from statsmodels.tsa.stattools import adfuller

In [93]:
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

'es_ES.UTF-8'

In [94]:
df = pd.read_csv('datos.csv')
df.head()

,Fecha,Consumo,Generación Solar,Temperatura
0,2023-01-01,109.934283,45.987795,25.979226
1,2023-01-02,97.234714,52.240925,20.108136
2,2023-01-03,112.953771,50.125924,27.041264
3,2023-01-04,130.460597,50.976761,16.487082
4,2023-01-05,95.316933,42.269902,30.145778


In [97]:
result = adfuller(df['Generación Solar'])
if result[1] > 0.05:
    print(f'La serie de tiempo {'Generación Solar'} no es estacionaria y que su valor de p-value es {result[1]:.8f}')
else:
    print(f'La serie de tiempo {'Generación Solar'} es estacionaria ya que su valor de p-value es {result[1]:.8f}')

La serie de tiempo Generación Solar es estacionaria ya que su valor de p-value es 0.00000000


,Fecha,Consumo,Generación Solar,Temperatura
0,2023-01-01,109.934283,45.987795,25.979226
1,2023-01-02,97.234714,52.240925,20.108136
2,2023-01-03,112.953771,50.125924,27.041264
3,2023-01-04,130.460597,50.976761,16.487082
4,2023-01-05,95.316933,42.269902,30.145778


In [74]:
px.line(df, x='Fecha', y=["Consumo", "Generación Solar"], title='Tendencia de Consumo y Generación Solar', line_shape="spline", labels={"value": "Energía (kWh)", "variable": "Tipo"})

In [75]:
px.line(df, x='Fecha', y='Generación Solar', title='Generación Energia Solar', line_shape="spline")

In [76]:
px.line(df, x='Fecha', y='Temperatura', title='Temperatura Energia',line_shape="spline")

In [77]:
df=df.rename(columns={'Fecha':'ds', 'Consumo':'y'})
#df.head()

In [78]:
px.line(df, x='ds', y='y', title='Consumo de Energía', labels={"value": "Energía (kWh)"})

In [79]:
model = Prophet()
model.fit(df)

15:25:27 - cmdstanpy - INFO - Chain [1] start processing
15:25:27 - cmdstanpy - INFO - Chain [1] done processing


In [80]:
future = model.make_future_dataframe(periods=5, freq = 'd')
#future.tail()

In [81]:
forecast = model.predict(future)

In [82]:
#forecast.tail()

In [83]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
365,2024-01-01,105.182938,79.563319,128.546063
366,2024-01-02,99.073764,73.709904,123.912720
367,2024-01-03,103.371946,78.788782,128.336978
368,2024-01-04,100.852021,76.363690,122.944436
369,2024-01-05,102.752490,77.247557,128.371551


In [84]:
px.line(forecast, x='ds', y='yhat', title='Predicción Consumo de Energía', labels={"value": "Energía (kWh)"})

In [85]:
#model.plot(forecast, xlabel='Fecha', ylabel='Consumo de Energía (kWh)')

In [86]:
df.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)

In [87]:
viz_df = df.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')
viz_df['yhat_exp'] = np.log(viz_df['yhat'])
viz_df['y_exp']=np.log(viz_df['y'])
viz_df.head()

,y,Generación Solar,Temperatura,yhat,yhat_lower,yhat_upper,yhat_exp,y_exp
ds,,,,,,,,
2023-01-01,109.934283,45.987795,25.979226,NaN,NaN,NaN,NaN,4.699883
2023-01-02,97.234714,52.240925,20.108136,NaN,NaN,NaN,NaN,4.577128
2023-01-03,112.953771,50.125924,27.041264,NaN,NaN,NaN,NaN,4.726979
2023-01-04,130.460597,50.976761,16.487082,NaN,NaN,NaN,NaN,4.871071
2023-01-05,95.316933,42.269902,30.145778,NaN,NaN,NaN,NaN,4.557207


In [88]:
px.line(viz_df, x=viz_df.index, y=['y_exp', 'yhat_exp'], title='Consumo de Energía', labels={"value": "Energía (kWh)"})